In [5]:
# 读取每篇论文的句子。

sents_papers = []
for i in doc_index_path[dir_i]:
    sents_papers_one = []
    onetext_open1 = 'duc_sents/' + str(dir_i) + '/duc' + str(i) + '.txt'
    temp_sentslist = open(onetext_open1,'r' ,encoding='utf-8')
    for sents in temp_sentslist:
        sents = sents.strip('\n')
        sents_papers_one.append(sents)
        
    sents_papers_one_dic = {}
    for k in range(len(sents_papers_one)):
        sents_papers_one_dic[k] = sents_papers_one[k]
        
    sents_papers.append(sents_papers_one_dic)

In [6]:
sents_papers_list = []
for i in range(len(sents_papers)):
    sents_papers_list_one = []
    for j in sents_papers[i].keys():
        sents_papers_list_one.append(sents_papers[i][j])
    sents_papers_list.append(sents_papers_list_one)

In [7]:
# sentence embedding:

pathdir1 = "./duc_sents_embeddings-DAN/" + str(dir_i) + ".npy"

temp_sents_embedding = np.load(pathdir1,allow_pickle=True)
sents_embedding_list = temp_sents_embedding.tolist()

In [8]:
papers_embeddings_list = []
for i in range(len(sents_embedding_list)):
    papers_embeddings_list.append(np.mean(sents_embedding_list[i], axis=0))
    
papers_embeddings = np.array(papers_embeddings_list)

In [ ]:
k_clu_list = [1,3,2]
# k-means的k值 = min(不重复文本向量的数量，specified-k)
doc_all = []
for item in doc_index_path[dir_i]:
    doc_all.append(item)
parent_children = {}
root_clusters = []
new_clusters = []
new_clusters.append(doc_all)
for clu_num in k_clu_list:
    temp_clu_num = clu_num
    categorys = []
    for item in new_clusters:
        item_str = [str(x) for x in item]
        parent_one = ','.join(item_str)
        parent_children[parent_one] = []
        papers_embeddings_onelist = []
        for d_index in item:
            papers_embeddings_onelist.append(papers_embeddings[d_index])
        papers_embeddings_one = np.array(papers_embeddings_onelist)
        unique_rows = np.vstack(list({tuple(row) for row in papers_embeddings_one}))
        if len(unique_rows) == 1:
            continue
        if len(unique_rows) < temp_clu_num: 
            km = KMeans(n_clusters=len(unique_rows)).fit(papers_embeddings_one)
        else:
            km = KMeans(n_clusters=temp_clu_num).fit(papers_embeddings_one)
        clusters = km.labels_.tolist()
        clusters_set = set()
        for item11 in clusters:
            clusters_set.add(item11)
        docindex_cluindex = {}
        for i in range(len(clusters)):
            docindex_cluindex[item[i]] = clusters[i]
        for i in range(len(clusters_set)):
            category_one = []
            for j in docindex_cluindex:
                if docindex_cluindex[j] == i:
                    category_one.append(j)
            categorys.append(category_one)
            
            category_one_str = [str(x) for x in category_one]
            child_one = ','.join(category_one_str)
            parent_children[parent_one].append(child_one)
    root_clusters.append(categorys)    
    new_clusters = []
    new_clusters = categorys

In [13]:
def order_bigsmall(strList):
    str_dic = {}
    for item in strList:
        str_dic[item] = len(item.split(','))
    ans_str_dic = sorted(str_dic.items(),key = lambda x:x[1],reverse = True)
    new_strList = []
    for item in ans_str_dic:
        new_strList.append(item[0])
    return new_strList

In [38]:
parent_seq = []
doc_all_str = [str(x) for x in doc_all]
root = ','.join(doc_all_str)
parent_seq.append(root)
child_old = []
for item0 in parent_children[root]:
    child_old.append(item0)
child_old = order_bigsmall(child_old)
while len(child_old) != 0:
    child_new = []
    for item1 in child_old:
        item1_list = item1.split(',')
        if len(item1_list) == 1:
            continue
        if item1 in parent_children:
            for word1 in parent_children[item1]:
                child_new.append(word1)
        parent_seq.append(item1)
    child_old = []
    child_old = order_bigsmall(child_new)

In [15]:
# a包含b。list类型
def np_becontained(a, b):
    contain_ornot = 0
    for word0 in b:
        if word0 not in a:
            contain_ornot = 1
            break
    if contain_ornot == 0:
        return True
    else:
        return False

In [16]:
# a包含b。
def str_becontained(a, b):
    a_list = a.split(',')
    b_list = b.split(',')
    contain_ornot = 0
    for word0 in b_list:
        if word0 not in a_list:
            contain_ornot = 1
            break
    if contain_ornot == 0:
        return True
    else:
        return False

In [17]:
# a和b是否有交集。有交集则返回True。list类型
def root_inter_ornot(a, b):
    inter_ornot = 0
    for word in a:
        if word in b:
            inter_ornot = 1
            break
    if inter_ornot == 1:
        return True
    else:
        return False

In [18]:
parent_tofather = {}
for item in parent_children:
    for c_item in parent_children[item]:
        parent_tofather[c_item] = item

parent_embedding = {}
for item in parent_seq:
    temp_papers_embedding = []
    temp_doc_list = item.split(',')
    for temp_doc in temp_doc_list:
        temp_papers_embedding.append(papers_embeddings[int(temp_doc)])
    temp_papers_embedding1 = np.array(temp_papers_embedding)
    parent_embedding[item] = np.mean(temp_papers_embedding1, axis=0)

In [23]:
parent_tobrother = {}
for item in parent_seq:
    bro_list = []
    item_list = item.split(',')
    for item1 in range(len(sents_papers)):
        if str(item1) not in item_list:
            bro_list.append(str(item1))
    if len(bro_list) != 0:
        parent_tobrother[item] = ','.join(bro_list)
        
brother_embedding = {}
for item in parent_tobrother:
    temp_brother_embedding = []
    item_bro_list = parent_tobrother[item].split(',')
    for bro_doc in item_bro_list:
        temp_brother_embedding.append(papers_embeddings[int(bro_doc)])
    temp_brother_embedding1 = np.array(temp_brother_embedding)
    brother_embedding[item] = np.mean(temp_brother_embedding1, axis=0)

In [27]:
sents_papers_tolist = []
for i in range(len(sents_papers)):
    sents_papers_tolist_one = {}
    for item in sents_papers[i]:
        sents_papers_tolist_one[item] = nltk.word_tokenize(sents_papers[i][item])
    sents_papers_tolist.append(sents_papers_tolist_one)

In [ ]:
words_sum_threshold = summary_length
summary_sents_index = []
selected_words_sum = 0

onefinish_ornot = 0
while onefinish_ornot != 1:
    for item0 in parent_seq:
        sents_inparent_score = {}
        item0_list = item0.split(',')
        doc_list = []
        for item1 in item0_list:
            doc_list.append(int(item1))

        item0_cen = parent_embedding[item0]

        for item2 in doc_list:
            for item3 in sents_papers[item2]:
                cos_simi = cosine_similarity(item0_cen.reshape(1,-1),sents_embedding_list[item2][item3].reshape(1,-1))
                cos_simil = cos_simi.tolist()
                score1 = cos_simil[0][0]
                
                score2 = 0
                for d_sent in summary_sents_index:
                    d_sent_list = d_sent.split(',')
                    cos2 = cosine_similarity(sents_embedding_list[int(d_sent_list[0])][int(d_sent_list[1])].reshape(1,-1),sents_embedding_list[item2][item3].reshape(1,-1))
                    cos21 = cos2.tolist()
                    cos211 = cos21[0][0]
                    if cos211 > score2:
                        score2 = cos211
                
                score3 = 0
                posi = math.exp((-1)*(item3+1)/(math.pow(len(sents_papers[item2]), 1.0/3)))
                score3 = max(0.5, posi)
                
                score4 = 0
                if item0 in parent_tofather:
                    cos4 = cosine_similarity(parent_embedding[parent_tofather[item0]].reshape(1,-1),sents_embedding_list[item2][item3].reshape(1,-1))
                    cos_simi4 = cos4.tolist()
                    score4 = cos_simi4[0][0]
                    
                score5 = 0
                if item0 in brother_embedding:
                    cos5 = cosine_similarity(brother_embedding[item0].reshape(1,-1),sents_embedding_list[item2][item3].reshape(1,-1))
                    cos_simi5 = cos5.tolist()
                    score5 = cos_simi5[0][0]
                    
                sents_inparent_score[str(item2) + "," + str(item3)] = 0.9*score1 + 0.1*(1-score5)

        index_max = 0
        ans_sents_inparent_score = sorted(sents_inparent_score.items(),key = lambda x:x[1],reverse = True)
        if ans_sents_inparent_score[0][1] == 0:
            continue
        for k in range(len(ans_sents_inparent_score)):
            if ans_sents_inparent_score[k][0] not in summary_sents_index:
                index_max = k
                break
        if ans_sents_inparent_score[index_max][1] != 0:
            summary_sents_index.append(ans_sents_inparent_score[index_max][0])
            temp_sent_index = ans_sents_inparent_score[index_max][0].split(',')
            selected_words_sum += len(sents_papers_tolist[int(temp_sent_index[0])][int(temp_sent_index[1])]) - 1
            if selected_words_sum > words_sum_threshold:
                onefinish_ornot = 1
                break

In [30]:
# 生成summary

summarys = []
for item in summary_sents_index:
    item_list = item.split(',')
    summarys.append(sents_papers[int(item_list[0])][int(item_list[1])])